In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.controller.combined_robot import *
from pkg.project_config import *

In [2]:
CONNECT_INDY = False

MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

In [3]:
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP),
    RobotConfig(1, RobotType.indy7, ((0.172,0,0.439), (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})]
              , connection_list=[False, CONNECT_INDY])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"kmb0": ((0,0,0), (0,0,0)), "indy1": ((0,0,0), (0,0,np.pi))}
# crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gscene.show_pose(-crob.home_pose)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
# gscene.show_pose(-crob.home_pose + ([0.2,0.2,np.pi/3,]+[0]*6))

## Set camera

In [5]:
from pkg.detector.camera.realsense import RealSense
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()
# aruco_map.print_markers()

# realsense = RealSense()
# realsense.initialize()
# camera_matrix, dist_coeffs = realsense.get_config()
# print("camera_matrix: {} \n {}".format(camera_matrix.shape, camera_matrix))
# print("dist_coeffs: {} \n {}".format(dist_coeffs.shape, dist_coeffs))

In [6]:
# img = realsense.get_image()
# pose_dict = aruco_map.get_object_pose_dict(img, camera_matrix, dist_coeffs)

## set workspace

In [7]:
gscene.set_workspace_boundary( -5, 5, -4, 4, -0.1, 1.75)

Please create a subscriber to the marker


## define robot and planning scene

In [8]:
from pkg.planning.scene import *
from demo_utils.environment import *

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.37,0.37,0.60)

MOBILE_NAME = "kmb0"
HOLD_LINK = "kmb0_platform"
HOLD_XYZ = (0.445 + 0.25, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)
HOLD_NAME = "hold0"

ROBOT_NAME = "indy1"
TOOL_LINK = "indy1_tcp"
TOOL_NAME = "brush_face"

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

gtems_robot = s_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
viewpoint = add_cam(gscene, tool_link=TOOL_LINK)
add_indy_tool_kiro(gscene, tool_link=TOOL_LINK, face_name=TOOL_NAME, zoff=-0.04)

pscene = PlanningScene(gscene, combined_robot=crob)

## planning pipeline

In [9]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)


from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


In [10]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.motion_filters = [GraspChecker(pscene)]

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Add action points

In [11]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [12]:


# table
gscene.create_safe(gtype=GEOTYPE.BOX, name="table", link_name="base_link",
                   dims=(2,0.7,0.5), center=(2,0,0.25), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)

# brush_face = pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, 
#                                   point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), rpy=(0,np.pi/2*1,0))



waypoints = [((2-0.5,-1,0),(0,0,np.pi/2)), ((2+0.5,-1,0),(0,0,np.pi/2)), ((2+0.5,1,0),(0,0,-np.pi/2)), ((2-0.5,1,0),(0,0,-np.pi/2))]
wp_list = []
for i_w, (xyz, rpy) in enumerate(waypoints):
    wp_list.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(i_w), link_name="base_link",
                           dims=WP_DIMS, center=xyz, 
                           rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, parent="floor")
    )
wp_task = pscene.create_subject(oname="waypoints", gname="floor", _type=WayopintTask, 
                                action_points_dict={wp.name: WayFrame(wp.name, wp, [0,0,0.05], [0,0,0]) for wp in wp_list})



## Add chair

In [13]:
chair_loc = (1, -1, CHAIR_DIM[2]/2)
obs_all = 3
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name=CHAIR_NAME, link_name="base_link",
                   dims=CHAIR_DIM, center=chair_loc, rpy=(0,0,0),
                   color=(1,1,0,1), display=True, collision=True, fixed=False)
chair = pscene.create_subject(oname=CHAIR_NAME, gname=CHAIR_NAME, _type=CylinderObject, 
                              GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                              GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
chair.action_points_dict[CHAIR_NAME+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}

## [Test] move chair to cover waypoints

In [14]:
initial_state = pscene.initialize_state(-crob.home_pose)
goal_nodes = [('floor',)+(4,)]

In [15]:
from pkg.planning.task.rrt_star import TaskRRTstar
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler


tplan = TaskRRTstar(pscene)
tplan.REWIND_MAX = 3
tplan.new_node_sampler = PenaltyNodeSampler(3, 3)
tplan.parent_node_sampler = UniformNodeSampler(3)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

mplan.reset_log(True)

ppline.search(initial_state, goal_nodes, verbose=True,
              timeout=0.5, timeout_loop=5, multiprocess=False,
              add_homing=3, home_pose=HOME_POSE, terminate_on_first=False)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_home = snode_schedule+ppline.add_return_motion(snode_schedule[-1], try_count=3, initial_state=initial_state)
ppline.play_schedule(snode_schedule_home)


try: 0 - ('floor', 0)->('floor', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('hold0', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('hold0', 0) = success
branching: 0->1 (0.09/5.0 s, steps/err: 48(78.8400173187 ms)/0.00101701847156)
try: 1 - ('hold0', 0)->('hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('hold0', 0)->('hold0', 1) = fail
try: 1 - ('hold0', 0)->('hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('hold0', 0)->('hold0', 1) = fail
try: 1 - ('hold0', 0)->('hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('hold0', 0)->('hold0', 1) = fail
try: 1 - ('hold0', 0)->('floor', 0)
try transition motion
transition motion tried: True
result: 1 - ('hold0', 0)->('floor', 0) = success
branching: 1->2 (0.18/5.0 s, steps/err: 69(74.5460987091 ms)/0.000558162651466)
try: 2 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 2 -

transition motion tried: True
result: 9 - ('floor', 1)->('hold0', 1) = success
branching: 9->19 (1.91/5.0 s, steps/err: 74(128.381967545 ms)/0.00194932979844)
try: 18 - ('hold0', 0)->('floor', 0)
try transition motion
transition motion tried: True
reserve 20 -> 3
result: 18 - ('hold0', 0)->('floor', 0) = success
branching: 18->20 (2.06/5.0 s, steps/err: 35(149.271011353 ms)/0.000495731481728)
try: 20 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
reserve 21 -> 19
reserve 21 -> 4
result: 20 - ('floor', 0)->('floor', 1) = success
branching: 20->21 (2.17/5.0 s, steps/err: 34(109.917879105 ms)/0.000794119831574)
try: 20 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
reserve 22 -> 19
reserve 22 -> 4
result: 20 - ('floor', 0)->('floor', 1) = success
branching: 20->22 (2.26/5.0 s, steps/err: 63(83.5118293762 ms)/0.001180773901)
try: 21 - ('floor', 1)->('floor', 2)
try transition motion
transition motion tried: True
reserve 23

transition motion tried: True
reserve 41 -> 19
reserve 41 -> 24
reserve 41 -> 25
reserve 41 -> 4
reserve 41 -> 23
result: 40 - ('floor', 0)->('floor', 1) = success
branching: 40->41 (3.83/5.0 s, steps/err: 95(142.905950546 ms)/0.00139188978185)
try: 40 - ('floor', 0)->('hold0', 0)
try transition motion
transition motion tried: True
reserve 42 -> 19
reserve 42 -> 24
reserve 42 -> 25
result: 40 - ('floor', 0)->('hold0', 0) = success
branching: 40->42 (3.92/5.0 s, steps/err: 29(86.4861011505 ms)/0.000980052702548)
try: 40 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
reserve 43 -> 19
reserve 43 -> 24
reserve 43 -> 25
reserve 43 -> 4
reserve 43 -> 23
result: 40 - ('floor', 0)->('floor', 1) = success
branching: 40->43 (4.03/5.0 s, steps/err: 90(112.969875336 ms)/0.000521415768175)
try: 40 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
reserve 44 -> 19
reserve 44 -> 24
reserve 44 -> 25
reserve 44 -> 4
reserve 44 -> 23
resul

In [16]:
mplan.update_gscene()

In [1]:
T_hold_ref = SE3(Rot_rpy((np.pi/2,0,-np.pi*11/12)), [-0.81,0.12,0.15])

NameError: name 'SE3' is not defined

In [2]:
# Qcur = HOME_POSE
Qcur = tplan.snode_dict[1].state.Q
traj, success = mplan.planner.plan_py("indy1", "indy1_tcp", np.concatenate(T2xyzquat(T_ie)), "indy1_link0", Qcur)
print(success)

NameError: name 'tplan' is not defined

In [3]:
gscene.show_motion(traj)

NameError: name 'gscene' is not defined

In [4]:
gscene.add_highlight_axis("ee","tar", "indy1_link0", center=T_ie[:3,3], orientation_mat=T_ie[:3,:3])

NameError: name 'gscene' is not defined

In [17]:
schedules

{10: [0, 1, 2, 3, 4, 5, 6, 9, 10],
 24: [0, 1, 13, 14, 16, 18, 24],
 25: [0, 1, 13, 14, 16, 19, 25],
 26: [0, 1, 13, 15, 17, 21, 26],
 27: [0, 1, 13, 15, 17, 22, 27],
 50: [0, 28, 29, 30, 32, 40, 50],
 54: [0, 28, 29, 30, 34, 43, 54],
 56: [0, 28, 29, 31, 37, 49, 56],
 63: [0, 28, 29, 30, 34, 41, 52, 62, 63],
 103: [0, 1, 65, 66, 71, 82, 103],
 107: [0, 1, 65, 67, 74, 85, 107],
 111: [0, 1, 65, 67, 74, 86, 111],
 115: [0, 1, 65, 68, 77, 91, 115],
 123: [0, 1, 65, 69, 80, 99, 123],
 127: [0, 1, 65, 69, 81, 100, 127],
 144: [0, 1, 65, 69, 80, 98, 119, 132, 133, 143, 144],
 150: [0, 28, 29, 31, 35, 45, 145, 146, 150],
 151: [0, 28, 29, 31, 35, 45, 145, 147, 151],
 152: [0, 28, 29, 31, 35, 45, 145, 148, 152],
 153: [0, 28, 29, 31, 35, 45, 145, 149, 153],
 168: [0, 1, 2, 154, 155, 156, 158, 162, 168],
 169: [0, 1, 2, 154, 155, 156, 158, 163, 169],
 170: [0, 1, 2, 154, 155, 156, 158, 164, 170],
 171: [0, 1, 2, 154, 155, 157, 160, 165, 171],
 172: [0, 1, 2, 154, 155, 157, 160, 166, 172],
 173

In [18]:
schedules_sorted

[[0, 28, 29, 31, 37, 49, 56],
 [0, 28, 29, 30, 32, 40, 50],
 [0, 28, 29, 30, 34, 43, 54],
 [0, 1, 65, 67, 74, 85, 107],
 [0, 1, 65, 67, 74, 86, 111],
 [0, 1, 65, 69, 81, 100, 127],
 [0, 1, 65, 69, 80, 99, 123],
 [0, 1, 65, 176, 177, 180, 189, 195, 201],
 [0, 1, 65, 176, 177, 178, 183, 191, 197],
 [0, 1, 65, 176, 177, 178, 183, 192, 198],
 [0, 1, 65, 176, 177, 180, 189, 193, 199],
 [0, 28, 29, 30, 34, 41, 52, 62, 63],
 [0, 1, 65, 176, 177, 180, 189, 194, 200],
 [0, 1, 65, 66, 71, 82, 103],
 [0, 28, 29, 134, 206, 209, 217, 228, 237],
 [0, 28, 29, 134, 206, 209, 217, 226, 235],
 [0, 1, 13, 15, 17, 22, 27],
 [0, 28, 29, 134, 206, 208, 215, 224, 233],
 [0, 1, 13, 15, 17, 21, 26],
 [0, 1, 65, 176, 177, 178, 183, 190, 196],
 [0, 28, 29, 134, 206, 209, 217, 227, 236],
 [0, 1, 65, 68, 77, 91, 115],
 [0, 28, 29, 134, 206, 208, 215, 223, 232],
 [0, 1, 65, 69, 80, 98, 119, 204, 205],
 [0, 1, 13, 14, 16, 19, 25],
 [0, 1, 13, 14, 16, 18, 24],
 [0, 1, 65, 241, 243, 247, 256, 270, 282],
 [0, 1, 65, 24

generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
generate table - PlanConditions
generate table - PlanList
button clicked
('floor', 0)->('floor', 0)
('floor', 0)->('hold0', 0)
button action done


In [15]:
# ppline.play_schedule(snode_schedule_home)

('floor', 0)->('floor', 0)
('floor', 0)->('hold0', 0)
('hold0', 0)->('floor', 0)
('floor', 0)->('floor', 1)
('floor', 1)->('floor', 2)
('floor', 2)->('floor', 3)
('floor', 3)->('floor', 4)
('floor', 4)->('floor', 4)


In [25]:
for node, sid_list in sorted(tplan.node_snode_dict.items()):
    print("node: {}".format(node))
    print(",".join(["{:3}, ".format(sidx) for sidx in sid_list]))
    print(",".join(["{:3}, ".format(tplan.snode_dict[sidx].depth) for sidx in sid_list]))

node: ('floor', 0)
  0, ,  2, 
  0, ,  2, 
node: ('floor', 1)
  3, 
  3, 
node: ('floor', 2)
  7, , 16, , 21, , 32, , 33, , 58, 
  4, ,  6, ,  6, ,  6, ,  6, ,  4, 
node: ('floor', 3)
  8, , 10, , 17, , 22, , 23, , 34, , 35, , 36, , 37, , 59, , 60, , 61, , 62, , 63, , 64, 
  5, ,  5, ,  7, ,  7, ,  7, ,  7, ,  7, ,  7, ,  7, ,  5, ,  5, ,  5, ,  5, ,  5, ,  5, 
node: ('floor', 4)
  9, , 11, , 12, , 18, , 20, , 24, , 25, , 26, , 27, , 28, , 29, , 38, , 39, , 40, , 41, , 43, , 44, , 45, , 46, , 48, , 49, , 50, , 51, , 53, , 54, , 55, , 56, , 67, , 68, , 69, , 70, , 71, , 72, , 73, , 81, , 82, , 83, , 84, , 85, , 86, , 87, , 95, , 96, , 97, , 98, , 99, ,100, ,101, ,109, ,110, ,111, ,112, ,113, ,114, ,115, ,123, 
  6, ,  6, ,  6, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, 

In [25]:
# from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream

# goal_pairs=[(CHAIR_NAME, 'goal'), (CHAIR_NAME[:-1]+"1", "floor"), (CHAIR_NAME[:-1]+"2", "floor")]
# TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS = 1, 100, 100, 30
# GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO = 100, 100, False, 100


# res, plan, log_dict = solve_in_pddlstream(pscene, mplan, MOBILE_NAME, HOLD_NAME, HOME_POSE, goal_pairs,
#                         TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS,
#                         GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO,
#                         use_pybullet_gui=False, USE_MOVEIT_IK=True)